# Building Your Predictor

The next step after preparing and importing your data via `Getting_Data_Ready.ipynb` is to build your first model.

The overall process for this is:

* Setup
* Create a Predictor
* Deploy a Predictor
* Obtain a Forecast

To get started, simply execute the cells below:


## Setup


Import the standard Python Libraries that are used in this lesson.

In [ ]:
import sys
import os
import time

import boto3

# importing forecast notebook utility from notebooks/common directory
#sys.path.insert( 0, os.path.abspath("../../common") )
import util

The line below will retrieve your shared variables from the first notebook.

In [ ]:
# %store -r

The last part of the setup process is to validate that your account can communicate with Amazon Forecast, the cell below does just that.

In [ ]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

session = boto3.Session(region_name=region) 
forecast = session.client(service_name='forecast') 
forecastquery = session.client(service_name='forecastquery')

In [ ]:
%store -r forecast_project_name
%store -r forecast_dataset_group_arn
%store -r forecast_dataset_arn
%store -r forecast_role_name
%store -r forecast_key
%store -r forecast_ds_import_job_arn

## Create a Predictor

Now in the previous notebook, your data was imported to be used by Forecast, here we will once again define your dataset information and then start building your model or predictor.

Forecast horizon is the number of number of time points to predicted in the future. For weekly data, a value of 12 means 12 weeks. Our example is hourly data, we try forecast the next day, so we can set to 24.

In [ ]:
predictor_name = forecast_project_name + '_deeparp_algo'

In [ ]:
forecast_horizon = 24

In [ ]:
algorithm_arn = 'arn:aws:forecast:::algorithm/Deep_AR_Plus'

In [ ]:
create_predictor_response=forecast.create_predictor(PredictorName=predictor_name, 
                                                    AlgorithmArn=algorithm_arn,
                                                    ForecastHorizon=forecast_horizon,
                                                    PerformAutoML= False,
                                                    PerformHPO=False,
                                                    EvaluationParameters= {"NumberOfBacktestWindows": 1, 
                                                                           "BackTestWindowOffset": 24}, 
                                                    InputDataConfig= {"DatasetGroupArn": forecast_dataset_group_arn},
                                                    FeaturizationConfig= {"ForecastFrequency": "H", 
                                                                          "Featurizations": 
                                                                          [
                                                                              {"AttributeName": "target_value", 
                                                                               "FeaturizationPipeline": 
                                                                                [
                                                                                  {"FeaturizationMethodName": "filling", 
                                                                                   "FeaturizationMethodParameters": 
                                                                                    {"frontfill": "none", 
                                                                                     "middlefill": "zero", 
                                                                                     "backfill": "zero"}
                                                                                  }
                                                                                ]
                                                                              }
                                                                          ]
                                                                         }
                                                 )

In [ ]:
forecast_predictor_arn=create_predictor_response['PredictorArn']

Check the status of the predictor. When the status change from **CREATE_IN_PROGRESS** to **ACTIVE**, we can continue to next steps. Depending on data size, model selection and hyper parameters，it can take 10 mins to more than one hour to be **ACTIVE**.

In [ ]:
status_indicator = util.StatusIndicator()

while True:
    status = forecast.describe_predictor(PredictorArn=forecast_predictor_arn)['Status']
    status_indicator.update(status)
    if status in ('ACTIVE', 'CREATE_FAILED'): break
    time.sleep(10)

status_indicator.end()

### Get Error Metrics

In [ ]:
forecast.get_accuracy_metrics(PredictorArn=forecast_predictor_arn)

## Create a Forecast

Now create a forecast using the model that was trained

In [ ]:
forecast_name = forecast_project_name + '_deeparp_algo_forecast'

In [ ]:
create_forecast_response=forecast.create_forecast(ForecastName=forecast_name,
                                                  PredictorArn=forecast_predictor_arn)
forecast_arn = create_forecast_response['ForecastArn']

Check the status of the forecast process, when the status change from **CREATE_IN_PROGRESS** to **ACTIVE**, we can continue to next steps. Depending on data size, model selection and hyper parameters，it can take 10 mins to more than one hour to be **ACTIVE**.

In [ ]:
status_indicator = util.StatusIndicator()

while True:
    status = forecast.describe_forecast(ForecastArn=forecast_arn)['Status']
    status_indicator.update(status)
    if status in ('ACTIVE', 'CREATE_FAILED'): break
    time.sleep(10)

status_indicator.end()

### Get Forecast

Once created, the forecast results are ready and you view them. 

In [ ]:
from pprint import pprint
print(forecast_arn)
print()
forecastResponse = forecastquery.query_forecast(
    ForecastArn=forecast_arn,
    Filters={"item_id":"client_12"}
)
pprint(forecastResponse)

## Next Steps

Now that your forecast has been created, to evaluate it use `3.Evaluating_Your_Predictor.ipynb` but before opening it, execute the cell below to share your variables again with the next notebook.

In [ ]:
%store forecast_arn
%store forecast_predictor_arn